In [1]:
import pandas as pd
import pandas as pd
import re
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from nltk.corpus import stopwords
from string import digits
import numpy as np
import nltk
from nltk.stem.porter import PorterStemmer
porter_stemmer = PorterStemmer()
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [2]:
def preprocess_data(s):
    s = s.lower()
    data = re.sub(r'[^\x00-\x7F]+', ' ', s)
    final_str = data.translate(None, string.punctuation)
    filter_str = final_str.translate(None, digits)
    nltk_tokens = nltk.word_tokenize(filter_str)
    #Next find the roots of the word
    str_= ''
    for w in nltk_tokens:

        if w not in stopwords.words('english'):
            str_ += ' '  + (lemmatizer.lemmatize(w))
    
    return str_.strip()

In [ ]:
def loadGloveModel(gloveFile):
    print "Loading Glove Model"
    f = open(gloveFile,'r')
    model = {}
    for line in f:
        splitLine = line.split()
        word = splitLine[0]
        embedding = np.array([float(val) for val in splitLine[1:]])
        model[word] = embedding
    print "Done.",len(model)," words loaded!"
    return model

model = loadGloveModel('/home/prashant/Downloads/kagsa/data/glove.840B.300d.txt')
#vocab = model.keys()

In [ ]:
train = pd.read_csv('/home/prashant/Downloads/c3cc8568-0-dataset/train.csv')
train.head(2)

In [ ]:
train['Consumer-complaint-summary'] = train['Consumer-complaint-summary'].apply(lambda x: " ".join(x.lower() for x in x.split()))

In [ ]:
train['Complaint-reason']  = train['Complaint-reason'].apply(preprocess_data)

In [ ]:
train['Consumer-complaint-summary'] = train['Consumer-complaint-summary'].str.replace('[^\w\s]','')

In [ ]:
train.head(2)

In [ ]:
from nltk.corpus import stopwords
stop = stopwords.words('english')

train['Consumer-complaint-summary'] = train['Consumer-complaint-summary'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))


In [ ]:
#Common word removal
freq = pd.Series(' '.join(train['Consumer-complaint-summary']).split()).value_counts()[:50]

In [ ]:
freq = list(freq.index)
train['Consumer-complaint-summary'] = train['Consumer-complaint-summary'].\
apply(lambda x: " ".join(x for x in x.split() if x not in freq))


In [ ]:
# freq = pd.Series(' '.join(train['Consumer-complaint-summary']).split()).value_counts()[-20000:]


In [ ]:
# freq = list(freq.index)
# train['Consumer-complaint-summary'] = train['Consumer-complaint-summary'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))


In [ ]:
train.head(2)

In [ ]:
train['Consumer-complaint-summary'] = train['Consumer-complaint-summary'].map(lambda x : re.sub("x+","",x))

In [ ]:
train['Consumer-complaint-summary'] = train['Consumer-complaint-summary'].map(lambda x : x.translate(None, digits))

In [ ]:
train.head()

In [ ]:
from textblob import Word
train['Consumer-complaint-summary'] = train['Consumer-complaint-summary'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))

In [ ]:
train.to_csv('final_prepro.csv',index=None)

In [ ]:

# tf_w_data = list(train['Consumer-complaint-summary'])
# tf_idf = TfidfVectorizer(max_features=5000)
# tf_idf_data = tf_idf.fit_transform(tf_w_data)
import pandas as pd
train = pd.read_csv('final_prepro.csv')

In [ ]:
# splitted = []
# for row in tf_w_data: 
#     splitted.append([word for word in row.split()])

In [ ]:
train_w2v = model

In [ ]:
def get_vector(model,all_data):
    vector_sen = []
    for d in all_data:
        single_sen_vec = []
        words = d.split(' ')
        vec_created =  0.0
        for w in words:
            try:
                get_word_vec = model[w]
                vec_created += 1.0
            except:
                pass
        if vec_created / len(words) > .5: 
            single_sen_vec.append(get_word_vec)
            v = np.array(single_sen_vec).mean(axis=0)
            vector_sen.append(v)
            sent =1
        else:
            vector_sen.append(np.zeros(300))
            sent = 0 
    return (vector_sen)

In [ ]:
embed_vec = get_vector(model,list(train['Consumer-complaint-summary']))

In [ ]:
# tf_w_data = []
# tf_idf_data = tf_idf_data.toarray()
# i = 0
# for row in splitted:
#     vec = [0 for i in range(50)]
    
#     temp_tfidf = []
#     for val in tf_idf_data[i]:
#         if val != 0:
#             temp_tfidf.append(val)
    
#     count = 0
#     tf_idf_sum = 0
#     for word in row:
#         try:
#             count += 1
#             tf_idf_sum = tf_idf_sum + temp_tfidf[count-1]
#             vec += (temp_tfidf[count-1] * train_w2v[word])
#         except:
#             pass
#     vec = (float)(1/tf_idf_sum) * vec
#     tf_w_data.append(vec)
#     i = i + 1


In [ ]:
train['merge'] = train["Consumer-complaint-summary"]+ " "+\
train["Complaint-reason"]

In [ ]:
train['merge']

In [ ]:
all_text = list(train['merge'])

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(sublinear_tf=True,use_idf=True, min_df=5, norm='l2', encoding='latin-1', ngram_range=(1, 2), stop_words='english')
tfidf.fit(all_text)


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(sublinear_tf=True,use_idf=True, min_df=5, norm='l2', encoding='latin-1', ngram_range=(1, 2), stop_words='english')
tfidf.fit(all_text)


In [ ]:
vec_cr = pd.DataFrame(tfidf.transform(all_text).toarray())
vec_cs = pd.DataFrame(embed_vec)

In [ ]:
comp_df = pd.concat([vec_cr,vec_cs],axis = 1)
print comp_df.shape
comp_df.head(2)

In [ ]:
features = comp_df
labels = train['Complaint-Status']

features.shape

In [ ]:
from sklearn.multiclass import OneVsRestClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
models = [
    (LinearSVC())

]
CV = 5
cv_df = pd.DataFrame(index=range(CV * len(models)))
entries = []
for model in models:
    model_name = model.__class__.__name__
    accuracies = cross_val_score(model, features, labels, scoring='f1_weighted', cv=CV)
    for fold_idx, accuracy in enumerate(accuracies):
        entries.append((model_name, fold_idx, accuracy))
        print (model_name, fold_idx, accuracy)
cv_df = pd.DataFrame(entries, columns=['model_name', 'fold_idx', 'accuracy'])